In [30]:
from __future__ import print_function
from time import time
import numpy as np
import pandas as pd
import os
import requests
import json
import numpy as np
import sklearn
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
from sklearn import svm
import torch
from keras.models import Sequential,Model
from keras.layers import Dense
from numpy import loadtxt

模型想法：


In [ ]:
ls

In [ ]:
ls ./csvFiles/

In [31]:
csvPath = "/home/ubuntu/Desktop/ADNI_MRI_Image/neuroScript/csvFiles"

In [17]:
volner = pd.read_csv(os.path.join(csvPath,"Freesurfer_S1200_Extensively_Processed_fMRI_Data.csv"))
volner = volner.dropna()
# MCI_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"MCI_amyloid_pos_freeStatsOut.csv"))
MCI_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"MCI_all_freeStatsOut.csv"))
MCI_freesurfer_Out = MCI_freesurfer_Out.dropna()
AD_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"AD_all_freeStatsOut.csv"))
AD_freesurfer_Out = AD_freesurfer_Out.dropna()
CN_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"CN_all_freeStatsOut.csv"))
CN_freesurfer_Out = CN_freesurfer_Out.dropna()
volner['Label'] = 0
AD_freesurfer_Out['Label'] = 1
MCI_freesurfer_Out['Label'] = 1
CN_freesurfer_Out['Label'] = 1

In [147]:
#此处的萎缩模型：此处认为amyloid以及tau均为阳性的为阳性，其余均为阴性
MCI_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"MCI_all_freeStatsOut.csv"))
MCI_freesurfer_Out = MCI_freesurfer_Out.dropna()
MCI_Within1Year = list(pd.read_csv(os.path.join(csvPath,"MCIWithin1Year.csv"))['PTID'])
MCI_freesurfer_Out_within1Year = MCI_freesurfer_Out[MCI_freesurfer_Out['Subject'].isin(MCI_Within1Year)]
ADNIMERGE_withTau_AbetaTauPosNeg = pd.read_csv(os.path.join(csvPath,'ADNIMERGE_withTau_AbetaTauPosNeg.csv'))
MCI_amyloid_tau_pos_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[ADNIMERGE_withTau_AbetaTauPosNeg['AbetaTauPosOrNeg'] == 1.0]['PTID'])

MCI_freesurfer_Out_within1Year['Label'] = 0
MCI_freesurfer_Out_within1Year.loc[MCI_freesurfer_Out_within1Year['Subject'].isin(MCI_amyloid_tau_pos_list),'Label'] = 1
################################################################################################################################
CN_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"CN_all_freeStatsOut.csv"))
CN_freesurfer_Out = CN_freesurfer_Out.dropna()
CN_Within1Year = list(pd.read_csv(os.path.join(csvPath,"CNWithin1Year.csv"))['PTID'])
CN_freesurfer_Out_within1Year = CN_freesurfer_Out[CN_freesurfer_Out['Subject'].isin(CN_Within1Year)]
ADNIMERGE_withTau_AbetaTauPosNeg = pd.read_csv(os.path.join(csvPath,'ADNIMERGE_withTau_AbetaTauPosNeg.csv'))
CN_amyloid_tau_pos_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[ADNIMERGE_withTau_AbetaTauPosNeg['AbetaTauPosOrNeg'] == 1.0]['PTID'])

CN_freesurfer_Out_within1Year['Label'] = 0
CN_freesurfer_Out_within1Year.loc[CN_freesurfer_Out_within1Year['Subject'].isin(CN_amyloid_tau_pos_list),'Label'] = 1
################################################################################################################################
AD_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"AD_all_freeStatsOut.csv"))
AD_freesurfer_Out = AD_freesurfer_Out.dropna()
AD_Within1Year = list(pd.read_csv(os.path.join(csvPath,"ADWithin1Year.csv"))['PTID'])
AD_freesurfer_Out_within1Year = AD_freesurfer_Out[AD_freesurfer_Out['Subject'].isin(AD_Within1Year)]
ADNIMERGE_withTau_AbetaTauPosNeg = pd.read_csv(os.path.join(csvPath,'ADNIMERGE_withTau_AbetaTauPosNeg.csv'))
AD_amyloid_tau_pos_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[ADNIMERGE_withTau_AbetaTauPosNeg['AbetaTauPosOrNeg'] == 1.0]['PTID'])

AD_freesurfer_Out_within1Year['Label'] = 0
AD_freesurfer_Out_within1Year.loc[AD_freesurfer_Out_within1Year['Subject'].isin(AD_amyloid_tau_pos_list),'Label'] = 1



/home/ubuntu/anaconda3/envs/spm/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19,20,21,104,105) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda3/envs/spm/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ubuntu/anaconda3/envs/spm/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

In [153]:
#此处的萎缩模型：此处认为amyloid以及tau均为阳性的为阳性，amyloid阴性(不论tau是否为阴性)的为阴性
MCI_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"MCI_all_freeStatsOut.csv"))
MCI_freesurfer_Out = MCI_freesurfer_Out.dropna()
MCI_Within1Year = list(pd.read_csv(os.path.join(csvPath,"MCIWithin1Year.csv"))['PTID'])
MCI_freesurfer_Out_within1Year = MCI_freesurfer_Out[MCI_freesurfer_Out['Subject'].isin(MCI_Within1Year)]
ADNIMERGE_withTau_AbetaTauPosNeg = pd.read_csv(os.path.join(csvPath,'ADNIMERGE_withTau_AbetaTauPosNeg.csv'))
MCI_amyloid_tau_pos_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[ADNIMERGE_withTau_AbetaTauPosNeg['AbetaTauPosOrNeg'] == 1.0]['PTID'])
MCI_amyloid_neg_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[(ADNIMERGE_withTau_AbetaTauPosNeg['AbetaPosOrNeg'] == 0.0) & (ADNIMERGE_withTau_AbetaTauPosNeg['VISCODE'] == 'bl')]['PTID'])

MCI_freesurfer_Out_within1Year['Label'] = 2
MCI_freesurfer_Out_within1Year.loc[MCI_freesurfer_Out_within1Year['Subject'].isin(MCI_amyloid_tau_pos_list),'Label'] = 1
MCI_freesurfer_Out_within1Year.loc[MCI_freesurfer_Out_within1Year['Subject'].isin(MCI_amyloid_neg_list),'Label'] = 0
MCI_freesurfer_Out_within1Year = MCI_freesurfer_Out_within1Year[MCI_freesurfer_Out_within1Year['Label'] != 2]
#####################################################################################################################
AD_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"AD_all_freeStatsOut.csv"))
AD_freesurfer_Out = AD_freesurfer_Out.dropna()
AD_Within1Year = list(pd.read_csv(os.path.join(csvPath,"ADWithin1Year.csv"))['PTID'])
AD_freesurfer_Out_within1Year = AD_freesurfer_Out[AD_freesurfer_Out['Subject'].isin(AD_Within1Year)]
ADNIMERGE_withTau_AbetaTauPosNeg = pd.read_csv(os.path.join(csvPath,'ADNIMERGE_withTau_AbetaTauPosNeg.csv'))
AD_amyloid_tau_pos_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[ADNIMERGE_withTau_AbetaTauPosNeg['AbetaTauPosOrNeg'] == 1.0]['PTID'])
AD_amyloid_neg_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[(ADNIMERGE_withTau_AbetaTauPosNeg['AbetaPosOrNeg'] == 0.0) & (ADNIMERGE_withTau_AbetaTauPosNeg['VISCODE'] == 'bl')]['PTID'])

AD_freesurfer_Out_within1Year['Label'] = 2
AD_freesurfer_Out_within1Year.loc[AD_freesurfer_Out_within1Year['Subject'].isin(AD_amyloid_tau_pos_list),'Label'] = 1
AD_freesurfer_Out_within1Year.loc[AD_freesurfer_Out_within1Year['Subject'].isin(AD_amyloid_neg_list),'Label'] = 0
AD_freesurfer_Out_within1Year = AD_freesurfer_Out_within1Year[AD_freesurfer_Out_within1Year['Label'] != 2]
#####################################################################################################################
CN_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"CN_all_freeStatsOut.csv"))
CN_freesurfer_Out = CN_freesurfer_Out.dropna()
CN_Within1Year = list(pd.read_csv(os.path.join(csvPath,"MCIWithin1Year.csv"))['PTID'])
CN_freesurfer_Out_within1Year = CN_freesurfer_Out[CN_freesurfer_Out['Subject'].isin(CN_Within1Year)]
ADNIMERGE_withTau_AbetaTauPosNeg = pd.read_csv(os.path.join(csvPath,'ADNIMERGE_withTau_AbetaTauPosNeg.csv'))
CN_amyloid_tau_pos_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[ADNIMERGE_withTau_AbetaTauPosNeg['AbetaTauPosOrNeg'] == 1.0]['PTID'])
CN_amyloid_neg_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[(ADNIMERGE_withTau_AbetaTauPosNeg['AbetaPosOrNeg'] == 0.0) & (ADNIMERGE_withTau_AbetaTauPosNeg['VISCODE'] == 'bl')]['PTID'])

CN_freesurfer_Out_within1Year['Label'] = 2
CN_freesurfer_Out_within1Year.loc[CN_freesurfer_Out_within1Year['Subject'].isin(CN_amyloid_tau_pos_list),'Label'] = 1
CN_freesurfer_Out_within1Year.loc[CN_freesurfer_Out_within1Year['Subject'].isin(CN_amyloid_neg_list),'Label'] = 0
CN_freesurfer_Out_within1Year = CN_freesurfer_Out_within1Year[CN_freesurfer_Out_within1Year['Label'] != 2]



/home/ubuntu/anaconda3/envs/spm/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ubuntu/anaconda3/envs/spm/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [159]:
#此处的萎缩模型：此处认为amyloid为阳性的则为阳性，amyloid阴性(不论tau是否为阴性)的为阴性
MCI_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"MCI_all_freeStatsOut.csv"))
MCI_freesurfer_Out = MCI_freesurfer_Out.dropna()
MCI_Within1Year = list(pd.read_csv(os.path.join(csvPath,"MCIWithin1Year.csv"))['PTID'])
MCI_freesurfer_Out_within1Year = MCI_freesurfer_Out[MCI_freesurfer_Out['Subject'].isin(MCI_Within1Year)]
ADNIMERGE_withTau_AbetaTauPosNeg = pd.read_csv(os.path.join(csvPath,'ADNIMERGE_withTau_AbetaTauPosNeg.csv'))
MCI_amyloid_pos_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[ADNIMERGE_withTau_AbetaTauPosNeg['AbetaPosOrNeg'] == 1.0]['PTID'])
MCI_amyloid_neg_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[(ADNIMERGE_withTau_AbetaTauPosNeg['AbetaPosOrNeg'] == 0.0) & (ADNIMERGE_withTau_AbetaTauPosNeg['VISCODE'] == 'bl')]['PTID'])

MCI_freesurfer_Out_within1Year['Label'] = 2
MCI_freesurfer_Out_within1Year.loc[MCI_freesurfer_Out_within1Year['Subject'].isin(MCI_amyloid_pos_list),'Label'] = 1
MCI_freesurfer_Out_within1Year.loc[MCI_freesurfer_Out_within1Year['Subject'].isin(MCI_amyloid_neg_list),'Label'] = 0
MCI_freesurfer_Out_within1Year = MCI_freesurfer_Out_within1Year[MCI_freesurfer_Out_within1Year['Label'] != 2]
#####################################################################################################################
AD_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"AD_all_freeStatsOut.csv"))
AD_freesurfer_Out = AD_freesurfer_Out.dropna()
AD_Within1Year = list(pd.read_csv(os.path.join(csvPath,"ADWithin1Year.csv"))['PTID'])
AD_freesurfer_Out_within1Year = AD_freesurfer_Out[AD_freesurfer_Out['Subject'].isin(AD_Within1Year)]
ADNIMERGE_withTau_AbetaTauPosNeg = pd.read_csv(os.path.join(csvPath,'ADNIMERGE_withTau_AbetaTauPosNeg.csv'))
AD_amyloid_pos_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[ADNIMERGE_withTau_AbetaTauPosNeg['AbetaPosOrNeg'] == 1.0]['PTID'])
AD_amyloid_neg_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[(ADNIMERGE_withTau_AbetaTauPosNeg['AbetaPosOrNeg'] == 0.0) & (ADNIMERGE_withTau_AbetaTauPosNeg['VISCODE'] == 'bl')]['PTID'])

AD_freesurfer_Out_within1Year['Label'] = 2
AD_freesurfer_Out_within1Year.loc[AD_freesurfer_Out_within1Year['Subject'].isin(AD_amyloid_pos_list),'Label'] = 1
AD_freesurfer_Out_within1Year.loc[AD_freesurfer_Out_within1Year['Subject'].isin(AD_amyloid_neg_list),'Label'] = 0
AD_freesurfer_Out_within1Year = AD_freesurfer_Out_within1Year[AD_freesurfer_Out_within1Year['Label'] != 2]
#####################################################################################################################
CN_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"CN_all_freeStatsOut.csv"))
CN_freesurfer_Out = CN_freesurfer_Out.dropna()
CN_Within1Year = list(pd.read_csv(os.path.join(csvPath,"MCIWithin1Year.csv"))['PTID'])
CN_freesurfer_Out_within1Year = CN_freesurfer_Out[CN_freesurfer_Out['Subject'].isin(CN_Within1Year)]
ADNIMERGE_withTau_AbetaTauPosNeg = pd.read_csv(os.path.join(csvPath,'ADNIMERGE_withTau_AbetaTauPosNeg.csv'))
CN_amyloid_pos_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[ADNIMERGE_withTau_AbetaTauPosNeg['AbetaPosOrNeg'] == 1.0]['PTID'])
CN_amyloid_neg_list = list(ADNIMERGE_withTau_AbetaTauPosNeg[(ADNIMERGE_withTau_AbetaTauPosNeg['AbetaPosOrNeg'] == 0.0) & (ADNIMERGE_withTau_AbetaTauPosNeg['VISCODE'] == 'bl')]['PTID'])

CN_freesurfer_Out_within1Year['Label'] = 2
CN_freesurfer_Out_within1Year.loc[CN_freesurfer_Out_within1Year['Subject'].isin(CN_amyloid_pos_list),'Label'] = 1
CN_freesurfer_Out_within1Year.loc[CN_freesurfer_Out_within1Year['Subject'].isin(CN_amyloid_neg_list),'Label'] = 0
CN_freesurfer_Out_within1Year = CN_freesurfer_Out_within1Year[CN_freesurfer_Out_within1Year['Label'] != 2]

/home/ubuntu/anaconda3/envs/spm/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ubuntu/anaconda3/envs/spm/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [138]:
MCI_freesurfer_Out_within1Year[MCI_freesurfer_Out_within1Year['Label'] == 0]

,Unnamed: 0,Subject,FS_L_ThalamusProper_Vol,FS_L_Caudate_Vol,FS_L_Putamen_Vol,FS_L_Pallidum_Vol,FS_L_Hippo_Vol,FS_L_Amygdala_Vol,FS_L_AccumbensArea_Vol,FS_R_ThalamusProper_Vol,...,FS_R_Superiorfrontal_Area,FS_R_Superiorparietal_Area,FS_R_Superiortemporal_Area,FS_R_Supramarginal_Area,FS_R_Frontalpole_Area,FS_R_Temporalpole_Area,FS_R_Transversetemporal_Area,FS_R_Insula_Area,FS_BrainSeg_Vol_No_Vent_Surf,Label
0,0,021_S_5099,5866.8,2909.7,3921.2,1555.8,3313.5,1317.4,440.0,5450.9,...,5812.0,4411.0,2897.0,3201.0,316.0,398.0,308.0,2097.0,9.353556e+05,0
4,4,141_S_6041,6592.0,3213.4,4930.7,1908.5,4378.0,1609.4,713.2,6693.3,...,7388.0,6066.0,4118.0,3823.0,269.0,482.0,427.0,2463.0,1.132295e+06,0
6,6,023_S_4115,6601.5,3356.6,4428.7,1976.0,4388.1,1825.1,464.1,5944.1,...,8302.0,5192.0,3611.0,3433.0,284.0,524.0,359.0,2748.0,1.138443e+06,0
7,7,006_S_6441,6943.2,3707.0,3944.6,1711.6,3309.2,1288.4,338.6,6809.9,...,6496.0,4839.0,3559.0,3697.0,246.0,427.0,296.0,2236.0,1.028701e+06,0
9,9,099_S_4202,6648.3,3720.5,4948.1,2153.8,3206.8,1374.1,546.3,6859.7,...,7582.0,5615.0,3951.0,4266.0,281.0,504.0,387.0,2718.0,1.132221e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,573,012_S_4094,5733.5,2576.8,3315.7,1392.6,3041.3,1395.4,269.1,5406.9,...,5563.0,4043.0,3022.0,2795.0,243.0,328.0,260.0,1737.0,8.373130e+05,0
575,575,021_S_2124,6473.3,3014.2,4161.7,1571.8,3204.1,1311.5,391.7,5789.4,...,7545.0,5734.0,3599.0,3208.0,343.0,463.0,324.0,2360.0,1.063252e+06,0
577,577,305_S_6742,6411.4,3145.8,4078.8,1660.2,2793.5,1067.9,225.1,5891.7,...,6379.0,4786.0,3116.0,2859.0,283.0,394.0,304.0,2420.0,1.066767e+06,0
579,579,067_S_4072,7150.1,3227.4,4603.1,1980.1,4404.9,1312.1,383.0,7052.9,...,8052.0,5171.0,4283.0,3763.0,323.0,463.0,369.0,2876.0,1.220189e+06,0


In [162]:
volAndCN_data = pd.concat([CN_freesurfer_Out, volner], axis=0, sort=False)
# volAndCN_data = volAndCN_data.dropna()
volAndMCI_data = pd.concat([MCI_freesurfer_Out, volner], axis=0, sort=False)
# volAndMCI_data = volAndMCI_data.dropna()
volAndAD_data = pd.concat([AD_freesurfer_Out, volner], axis=0, sort=False)
# volAndAD_data = volAndAD_data.dropna()
AD_MCI_freesurfer_Out_within1Year = pd.concat([AD_freesurfer_Out_within1Year, MCI_freesurfer_Out_within1Year], axis=0, sort=False)



In [19]:
volAndCN_data

,Unnamed: 0,Subject,FS_L_ThalamusProper_Vol,FS_L_Caudate_Vol,FS_L_Putamen_Vol,FS_L_Pallidum_Vol,FS_L_Hippo_Vol,FS_L_Amygdala_Vol,FS_L_AccumbensArea_Vol,FS_R_ThalamusProper_Vol,...,FS_L_WM_Hypointens_Vol,FS_R_WM_Hypointens_Vol,FS_L_Non-WM_Hypointens_Vol,FS_R_Non-WM_Hypointens_Vol,FS_OpticChiasm_Vol,FS_CC_Posterior_Vol,FS_CC_MidPosterior_Vol,FS_CC_Central_Vol,FS_CC_MidAnterior_Vol,FS_CC_Anterior_Vol
0,0.0,037_S_4071,7524.5,4313.9,4709.2,2411.9,4141.2,1453.2,303.6,7778.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,018_S_4400,7040.7,2889.0,4192.5,1805.1,3917.6,1660.1,390.7,6533.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,029_S_4290,6034.1,3711.6,4222.0,2110.8,3113.6,1434.0,439.5,5530.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,029_S_4652,6008.3,2884.1,3715.7,1612.7,3478.3,1190.9,457.1,5293.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,031_S_4474,6948.1,3291.7,3953.5,1728.6,3193.5,1227.4,469.7,6342.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,NaN,992774,8416.0,3756.0,5080.0,1188.0,4250.0,1473.0,538.0,6862.0,...,0.0,0.0,0.0,0.0,224.0,971.0,458.0,416.0,415.0,1004.0
1202,NaN,993675,7477.0,3449.0,4874.0,1054.0,4571.0,1358.0,564.0,6879.0,...,0.0,0.0,0.0,0.0,107.0,702.0,269.0,306.0,319.0,660.0
1203,NaN,994273,8416.0,4236.0,5961.0,1364.0,4159.0,1627.0,700.0,8228.0,...,0.0,0.0,0.0,0.0,232.0,908.0,339.0,368.0,414.0,741.0
1204,NaN,995174,9334.0,4546.0,6769.0,1497.0,4616.0,1629.0,535.0,8032.0,...,0.0,0.0,0.0,0.0,142.0,765.0,441.0,458.0,424.0,863.0


In [6]:
features = ['FS_L_ThalamusProper_Vol', 'FS_L_Caudate_Vol', 'FS_L_Putamen_Vol', 'FS_L_Pallidum_Vol', 
            'FS_L_Hippo_Vol', 'FS_L_Amygdala_Vol', 'FS_L_AccumbensArea_Vol', 'FS_R_ThalamusProper_Vol', 
            'FS_R_Caudate_Vol', 'FS_R_Putamen_Vol', 'FS_R_Pallidum_Vol', 'FS_R_Hippo_Vol', 'FS_R_Amygdala_Vol', 
            'FS_R_AccumbensArea_Vol', 'FS_L_Bankssts_Thck', 'FS_L_Caudalanteriorcingulate_Thck', 
            'FS_L_Caudalmiddlefrontal_Thck', 'FS_L_Cuneus_Thck', 'FS_L_Entorhinal_Thck', 'FS_L_Fusiform_Thck', 
            'FS_L_Inferiorparietal_Thck', 'FS_L_Inferiortemporal_Thck', 'FS_L_Isthmuscingulate_Thck', 
            'FS_L_Lateraloccipital_Thck', 'FS_L_Lateralorbitofrontal_Thck', 'FS_L_Lingual_Thck', 
            'FS_L_Medialorbitofrontal_Thck', 'FS_L_Middletemporal_Thck', 'FS_L_Parahippocampal_Thck', 
            'FS_L_Paracentral_Thck', 'FS_L_Parsopercularis_Thck', 'FS_L_Parsorbitalis_Thck', 
            'FS_L_Parstriangularis_Thck', 'FS_L_Pericalcarine_Thck', 'FS_L_Postcentral_Thck', 
            'FS_L_Posteriorcingulate_Thck', 'FS_L_Precentral_Thck', 'FS_L_Precuneus_Thck', 
            'FS_L_Rostralanteriorcingulate_Thck', 'FS_L_Rostralmiddlefrontal_Thck', 'FS_L_Superiorfrontal_Thck', 
            'FS_L_Superiorparietal_Thck', 'FS_L_Superiortemporal_Thck', 'FS_L_Supramarginal_Thck', 
            'FS_L_Frontalpole_Thck', 'FS_L_Temporalpole_Thck', 'FS_L_Transversetemporal_Thck', 
            'FS_L_Insula_Thck', 'FS_R_Bankssts_Thck', 'FS_R_Caudalanteriorcingulate_Thck', 
            'FS_R_Caudalmiddlefrontal_Thck', 'FS_R_Cuneus_Thck', 'FS_R_Entorhinal_Thck', 'FS_R_Fusiform_Thck', 
            'FS_R_Inferiorparietal_Thck', 'FS_R_Inferiortemporal_Thck', 'FS_R_Isthmuscingulate_Thck', 
            'FS_R_Lateraloccipital_Thck', 'FS_R_Lateralorbitofrontal_Thck', 'FS_R_Lingual_Thck', 
            'FS_R_Medialorbitofrontal_Thck', 'FS_R_Middletemporal_Thck', 'FS_R_Parahippocampal_Thck', 
            'FS_R_Paracentral_Thck', 'FS_R_Parsopercularis_Thck', 'FS_R_Parsorbitalis_Thck', 
            'FS_R_Parstriangularis_Thck', 'FS_R_Pericalcarine_Thck', 'FS_R_Postcentral_Thck', 
            'FS_R_Posteriorcingulate_Thck', 'FS_R_Precentral_Thck', 'FS_R_Precuneus_Thck', 
            'FS_R_Rostralanteriorcingulate_Thck', 'FS_R_Rostralmiddlefrontal_Thck', 
            'FS_R_Superiorfrontal_Thck', 'FS_R_Superiorparietal_Thck', 'FS_R_Superiortemporal_Thck',
            'FS_R_Supramarginal_Thck', 'FS_R_Frontalpole_Thck', 'FS_R_Temporalpole_Thck', 
            'FS_R_Transversetemporal_Thck', 'FS_R_Insula_Thck']

In [28]:
##Convert pd.Dataframe to feature(np.array) here ## with label here
def Convert2FeatureVectorWithLabel(pdFrame):
    subject = []
    pdFrame['reverse_FS_BrainSeg_Vol_No_Vent_Surf'] = 1.0 / pdFrame['FS_BrainSeg_Vol_No_Vent_Surf']
    X = []
    x =[]
    weighted_mean_feature_dict = {}
    for feature in features:
        if feature.split('_')[-1] == 'Thck':
            feature_area = feature[0:-4] + 'Area'
            weighted_mean_feature_dict[feature] = (pdFrame[feature] * pdFrame[feature_area]).sum() / pdFrame[feature_area].sum()
        if feature.split('_')[-1] == 'Vol':
            weighted_mean_feature_dict[feature] = (pdFrame[feature] * pdFrame['reverse_FS_BrainSeg_Vol_No_Vent_Surf']).sum() / pdFrame['reverse_FS_BrainSeg_Vol_No_Vent_Surf'].sum()   
    variance_feature_dict = {}
    for feature in features:
        if feature.split('_')[-1] == 'Thck':
            feature_area = feature[0:-4] + 'Area'
            variance_feature_dict[feature] = np.sqrt((pdFrame[feature_area] * np.square(pdFrame[feature] - weighted_mean_feature_dict[feature])).sum() / (pdFrame[feature_area].sum() - (np.square(pdFrame[feature_area])).sum() / pdFrame[feature_area].sum()))
        if feature.split('_')[-1] == 'Vol':
            variance_feature_dict[feature] = np.sqrt((pdFrame["reverse_FS_BrainSeg_Vol_No_Vent_Surf"] * np.square(pdFrame[feature] - weighted_mean_feature_dict[feature])).sum() / (pdFrame["reverse_FS_BrainSeg_Vol_No_Vent_Surf"].sum() - (np.square(pdFrame["reverse_FS_BrainSeg_Vol_No_Vent_Surf"])).sum() / pdFrame["reverse_FS_BrainSeg_Vol_No_Vent_Surf"].sum()))
    for index,row in pdFrame.iterrows():
        for feature in features:
            x.append((np.float(row[feature]) - weighted_mean_feature_dict[feature]) / variance_feature_dict[feature])
#         x.append(row['Subject'])
        subject.append(row['Subject'])
        x.append(row['Label'])
        X.append(x)
        x =[]
        if len(X) % 100 == 0:
            pass
#             print("finish 100")
    X = np.array(X)
    return X, subject


In [146]:
Vector_with_Label, Vector_subject = Convert2FeatureVectorWithLabel(MCI_freesurfer_Out_within1Year)
print(Vector_with_Label.shape)

print(Vector_with_Label[:,0:-1].shape,Vector_with_Label[:,-1].shape)
X = Vector_with_Label[:,0:-1]
y = Vector_with_Label[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(6,2), random_state=1,max_iter=700)
clf.fit(X_train,y_train)
clf.score(X_test, y_test)

finish 100
finish 100
finish 100
(394, 83)
(394, 82) (394,)


0.6134453781512605

In [163]:
Vector_with_Label, Vector_subject = Convert2FeatureVectorWithLabel(AD_MCI_freesurfer_Out_within1Year)
print(Vector_with_Label.shape)
print(Vector_with_Label[:,0:-1].shape,Vector_with_Label[:,-1].shape)
X = Vector_with_Label[:,0:-1]
y = Vector_with_Label[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
# define the keras model
model = Sequential()
model.add(Dense(16, input_dim=82, activation='relu'))
model.add(Dense(4, activation='relu',name = 'my_layer'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=10, batch_size=5)
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
(729, 83)
(729, 82) (729,)
Epoch 1/10
510/510 [==============================] - 0s 771us/step - loss: 0.7380 - accuracy: 0.5176
Epoch 2/10
510/510 [==============================] - 0s 364us/step - loss: 0.6598 - accuracy: 0.6059
Epoch 3/10
510/510 [==============================] - 0s 331us/step - loss: 0.6198 - accuracy: 0.6608
Epoch 4/10
510/510 [==============================] - 0s 332us/step - loss: 0.5961 - accuracy: 0.6902
Epoch 5/10
510/510 [==============================] - 0s 330us/step - loss: 0.5743 - accuracy: 0.7235
Epoch 6/10
510/510 [==============================] - 0s 352us/step - loss: 0.5550 - accuracy: 0.7412
Epoch 7/10
510/510 [==============================] - 0s 307us/step - loss: 0.5382 - accuracy: 0.7451
Epoch 8/10
510/510 [==============================] - 0s 343us/step - loss: 0.5202 - accuracy: 0.7588
Epoch 9/10
510/510 [==============================] - 0s 330us/step - loss: 0.49

In [25]:
def clusterFeatureExtractor(data_with_volner,data_without_volner):
    Vector_with_Label, Vector_subject = Convert2FeatureVectorWithLabel(data_with_volner)
    Vector_without_volner_with_Label, Vector_without_volner_Vector_subject = Convert2FeatureVectorWithLabel(data_without_volner)
    print(Vector_with_Label.shape)
    print(Vector_with_Label[:,0:-1].shape,Vector_with_Label[:,-1].shape)
    X = Vector_with_Label[:,0:-1]
    y = Vector_with_Label[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    # define the keras model
    model = Sequential()
    model.add(Dense(12, input_dim=82, activation='relu'))
    model.add(Dense(6, activation='relu',name = 'my_layer'))
    model.add(Dense(1, activation='sigmoid'))
    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit the keras model on the dataset
    model.fit(X_train, y_train, epochs=15, batch_size=10)
    # evaluate the keras model
    _, accuracy = model.evaluate(X_test, y_test)
    print('Accuracy: %.2f' % (accuracy*100))
    layer_name = 'my_layer'
    intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    intermediate_output = intermediate_layer_model.predict(Vector_without_volner_with_Label[:,0:-1])
    print(intermediate_output.shape)
    return intermediate_output

In [27]:
CN_intermediate_output = clusterFeatureExtractor(volAndCN_data,CN_freesurfer_Out)
MCI_intermediate_output = clusterFeatureExtractor(volAndMCI_data,MCI_freesurfer_Out)
AD_intermediate_output = clusterFeatureExtractor(volAndAD_data,AD_freesurfer_Out)

finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
(1255, 83)
(1255, 82) (1255,)
Epoch 1/15
878/878 [==============================] - 0s 312us/step - loss: 0.7669 - accuracy: 0.4806
Epoch 2/15
878/878 [==============================] - 0s 131us/step - loss: 0.3288 - accuracy: 0.9339
Epoch 3/15
878/878 [==============================] - 0s 137us/step - loss: 0.1581 - accuracy: 0.9841
Epoch 4/15
878/878 [==============================] - 0s 129us/step - loss: 0.0948 - accuracy: 0.9886
Epoch 5/15
878/878 [==============================] - 0s 134us/step - loss: 0.0600 - accuracy: 0.9966
Epoch 6/15
878/878 [==============================] - 0s 130us/step - loss: 0.0403 - accuracy: 0.9977
Epoch 7/15
878/878 [==============================] - 0s 139us/step - loss: 0.0286 - accuracy: 0.9989
Epoch 8/15
878/878 [==============================] - 0s 133us/step - loss: 0.0203 - accuracy: 0.9989
Epoch 9/15


In [ ]:
##Build my input for clustering
layer_name = 'my_layer'
intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(MCI_vector)
print(intermediate_output.shape)

# Clustering methods

In [31]:
clusterInput = MCI_intermediate_output

In [33]:
len(clusterInput[0])

6

### K-means methods

In [50]:
np.random.seed(42)
kmeans = KMeans(n_clusters=3, random_state=0).fit(clusterInput)

In [51]:
kmeans.labels_

array([1, 2, 2, 1, 0, 1, 2, 0, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1,
       0, 2, 0, 1, 1, 2, 1, 0, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 0, 2, 1, 0,
       1, 0, 2, 0, 0, 2, 2, 1, 0, 1, 2, 1, 1, 0, 2, 0, 0, 1, 1, 1, 1, 2,
       1, 0, 1, 2, 0, 1, 2, 0, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 0, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 0, 1, 2, 0, 1, 0,
       0, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 0, 0, 0, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 0, 1, 2, 1, 1, 1, 1, 0, 2, 0,
       1, 1, 2, 1, 1, 1, 0, 2, 1, 0, 2, 2, 1, 2, 0, 0, 1, 0, 0, 1, 0, 2,
       0, 1, 1, 2, 1, 1, 1, 2, 1, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 2, 0, 0, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 0, 1, 2, 2, 2, 1, 0, 0, 1,
       0, 1, 2, 1, 1, 0, 1, 0, 1, 0, 2, 1, 2, 1, 1,

### UMAP methods

### Spectral Clustering

## This is the start of draft

In [ ]:
np.savetxt(os.path.join(csvPath,"MCI_sne.tsv"),MCI_vector,delimiter='\t')

In [ ]:
np.savetxt(os.path.join(csvPath,"all_data.tsv"),Vector_with_Label[:,0:-1],delimiter='\t')

In [ ]:
MCI_vector,MCI_subject = Convert2FeatureVector(MCI_freesurfer_Out)
volner_vector, volner_subject = Convert2FeatureVector(volner)

MCI_label = np.ones(302)
volner_label = np.zeros(1113)
print(MCI_label.shape,volner_label.shape)
y = np.concatenate((MCI_label,volner_label))
print(y.shape)

X = np.concatenate((MCI_vector,volner_vector))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(64,128), random_state=1,max_iter=700)
clf.fit(X_train,y_train)
clf.score(X_test, y_test)

## Keras tutorial

In [ ]:
# load the dataset
dataset = loadtxt(os.path.join(csvPath,'pima-indians-diabetes.data.csv'), delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# load the dataset
dataset = loadtxt(os.path.join(csvPath,'pima-indians-diabetes.data.csv'), delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, y, epochs=15, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

## This is the start of draft

In [ ]:
##Convert pd.Dataframe to feature(np.array) here ## No label here
def Convert2FeatureVector(pdFrame):
    subject = []
    pdFrame = pdFrame.dropna()
    pdFrame['reverse_FS_BrainSeg_Vol_No_Vent_Surf'] = 1.0 / pdFrame['FS_BrainSeg_Vol_No_Vent_Surf']
    X = []
    x =[]
    weighted_mean_feature_dict = {}
    for feature in features:
        if feature.split('_')[-1] == 'Thck':
            feature_area = feature[0:-4] + 'Area'
            weighted_mean_feature_dict[feature] = (pdFrame[feature] * pdFrame[feature_area]).sum() / pdFrame[feature_area].sum()
        if feature.split('_')[-1] == 'Vol':
            weighted_mean_feature_dict[feature] = (pdFrame[feature] * pdFrame['reverse_FS_BrainSeg_Vol_No_Vent_Surf']).sum() / pdFrame['reverse_FS_BrainSeg_Vol_No_Vent_Surf'].sum()   
    variance_feature_dict = {}
    for feature in features:
        if feature.split('_')[-1] == 'Thck':
            feature_area = feature[0:-4] + 'Area'
            variance_feature_dict[feature] = np.sqrt((pdFrame[feature_area] * np.square(pdFrame[feature] - weighted_mean_feature_dict[feature])).sum() / (pdFrame[feature_area].sum() - (np.square(pdFrame[feature_area])).sum() / pdFrame[feature_area].sum()))
        if feature.split('_')[-1] == 'Vol':
            variance_feature_dict[feature] = np.sqrt((pdFrame["reverse_FS_BrainSeg_Vol_No_Vent_Surf"] * np.square(pdFrame[feature] - weighted_mean_feature_dict[feature])).sum() / (pdFrame["reverse_FS_BrainSeg_Vol_No_Vent_Surf"].sum() - (np.square(pdFrame["reverse_FS_BrainSeg_Vol_No_Vent_Surf"])).sum() / pdFrame["reverse_FS_BrainSeg_Vol_No_Vent_Surf"].sum()))
    for index,row in pdFrame.iterrows():
        for feature in features:
            x.append((np.float(row[feature]) - weighted_mean_feature_dict[feature]) / variance_feature_dict[feature])
#         x.append(row['Subject'])
        X.append(x)
        subject.append(row['Subject'])
        x =[]
        if len(X) % 100 == 0:
            print("finish 100")
    X = np.array(X)
    return X, subject


In [ ]:
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(64,100), random_state=1,max_iter=700).fit(X_train, y_train)
clf.score(X_test, y_test)


In [ ]:
volner_no_Nan = volner.dropna()

In [ ]:
weighted_mean_feature_dict

In [ ]:
weighted_mean_feature_dict = {}
for feature in features:
    if feature.split('_')[-1] == 'Thck':
        feature_area = feature[0:-4] + 'Area'
        weighted_mean_feature_dict[feature] = (volner_no_Nan[feature] * volner_no_Nan[feature_area]).sum() / volner_no_Nan[feature_area].sum()
    if feature.split('_')[-1] == 'Vol':
        weighted_mean_feature_dict[feature] = (volner_no_Nan[feature] * volner_no_Nan['reverse_FS_BrainSeg_Vol_No_Vent_Surf']).sum() / volner_no_Nan['reverse_FS_BrainSeg_Vol_No_Vent_Surf'].sum()   
    

In [ ]:
variance_feature_dict = {}
for feature in features:
    if feature.split('_')[-1] == 'Thck':
        feature_area = feature[0:-4] + 'Area'
        variance_feature_dict[feature] = np.sqrt((volner_no_Nan[feature_area] * np.square(volner_no_Nan[feature] - weighted_mean_feature_dict[feature])).sum() / (volner_no_Nan[feature_area].sum() - (np.square(volner_no_Nan[feature_area])).sum() / volner_no_Nan[feature_area].sum()))
    if feature.split('_')[-1] == 'Vol':
        variance_feature_dict[feature] = np.sqrt((volner_no_Nan["reverse_FS_BrainSeg_Vol_No_Vent_Surf"] * np.square(volner_no_Nan[feature] - weighted_mean_feature_dict[feature])).sum() / (volner_no_Nan["reverse_FS_BrainSeg_Vol_No_Vent_Surf"].sum() - (np.square(volner_no_Nan["reverse_FS_BrainSeg_Vol_No_Vent_Surf"])).sum() / volner_no_Nan["reverse_FS_BrainSeg_Vol_No_Vent_Surf"].sum()))


In [ ]:
volner_no_Nan['reverse_FS_BrainSeg_Vol_No_Vent_Surf'] = 1.0 / volner_no_Nan['FS_BrainSeg_Vol_No_Vent_Surf']

In [ ]:
variance_feature_dict

In [ ]:
X = []
x =[]
for index,row in volner_no_Nan.iterrows():
    for feature in features:
        x.append((np.float(row[feature]) - weighted_mean_feature_dict[feature]) / variance_feature_dict[feature])
    X.append(x)
    x =[]
    if len(X) % 100 == 0:
        print("finish 100")
X = np.array(X)      
print(X.shape)

In [ ]:
selectFeatures2 = ["left-wthi-bankssts","left-wthi-caudalanteriorcingulate","left-wthi-caudalmiddlefrontal",
                  "left-wthi-cuneus","left-wthi-entorhinal","left-wthi-fusiform","left-wthi-inferiorparietal",
                  "left-wthi-inferiortemporal","left-wthi-isthmuscingulate","left-wthi-lateraloccipital",
                  "left-wthi-lateralorbitofrontal","left-wthi-lingual","left-wthi-medialorbitofrontal",
                  "left-wthi-middletemporal","left-wthi-parahippocampal","left-wthi-paracentral",
                  "left-wthi-parsopercularis","left-wthi-parsorbitalis","left-wthi-parstriangularis",
                  "left-wthi-pericalcarine","left-wthi-postcentral","left-wthi-posteriorcingulate",
                  "left-wthi-precentral","left-wthi-precuneus","left-wthi-rostralanteriorcingulate",
                  "left-wthi-rostralmiddlefrontal","left-wthi-superiorfrontal","left-wthi-superiorparietal",
                  "left-wthi-superiortemporal","left-wthi-supramarginal","left-wthi-frontalpole",
                  "left-wthi-temporalpole","left-wthi-transversetemporal","left-wthi-insula","right-wthi-bankssts",
                  "right-wthi-caudalanteriorcingulate","right-wthi-caudalmiddlefrontal","right-wthi-cuneus",
                  "right-wthi-entorhinal","right-wthi-fusiform","right-wthi-inferiorparietal",
                  "right-wthi-inferiortemporal","right-wthi-isthmuscingulate","right-wthi-lateraloccipital",
                  "right-wthi-lateralorbitofrontal","right-wthi-lingual","right-wthi-medialorbitofrontal",
                  "right-wthi-middletemporal","right-wthi-parahippocampal","right-wthi-paracentral",
                  "right-wthi-parsopercularis","right-wthi-parsorbitalis","right-wthi-parstriangularis",
                  "right-wthi-pericalcarine","right-wthi-postcentral","right-wthi-posteriorcingulate",
                  "right-wthi-precentral","right-wthi-precuneus","right-wthi-rostralanteriorcingulate",
                  "right-wthi-rostralmiddlefrontal","right-wthi-superiorfrontal","right-wthi-superiorparietal",
                  "right-wthi-superiortemporal","right-wthi-supramarginal","right-wthi-frontalpole",
                  "right-wthi-temporalpole","right-wthi-transversetemporal","right-wthi-insula",
                  "right-svol-Thalamus","right-svol-Caudate","right-svol-Putamen","right-svol-Pallidum",
                  "right-svol-Hippocampus","right-svol-Amygdala","right-svol-Accumbens","left-svol-Thalamus",
                  "left-svol-Caudate","left-svol-Putamen","left-svol-Pallidum","left-svol-Hippocampus",
                  "left-svol-Amygdala","left-svol-Accumbens"]

In [ ]:
#load the jwt token (run bl login to create this file)
jwt_file = open(os.environ['HOME']+'/.config/brainlife.io/.jwt', mode='r')
jwt = jwt_file.read()
#query datasets records
find = { 
    '_group_id': '69', #see project detail page 
    'service': 'brainlife/app-freesurfer', 
    #'service_branch': '0.0.5',
    'status': 'finished' }
params = { 
    'limit': 100, 
    'select': 'config._inputs.meta', #we just want product and meta
    'find': json.JSONEncoder().encode(find) }
res = requests.get('https://brainlife.io/api/amaretti/task', params=params, headers={'Authorization': 'Bearer '+jwt})
if res.status_code != 200:
    raise Exception("failed to download datasets list:"+res.status_code)
tasks = res.json()["tasks"]
for task in tasks:
    taskid=task["_id"]
    subject=task["config"]["_inputs"][0]["meta"]["subject"]
    print(taskid, subject)
    url = 'https://brainlife.io/api/amaretti/task/download/'+taskid+'/freesurfer/output/stats?at='+jwt
    res = requests.get(url, allow_redirects=True)
    open(subject+'.tar.gz', 'wb').write(res.content)

In [ ]:
res = requests.get('https://brainlife.io/api/amaretti/resource',headers={'Authorization': 'Bearer '+jwt})

In [ ]:
#This verison is for all volume feature
selectFeatures1 = ["left-vol-bankssts","left-vol-caudalanteriorcingulate","left-vol-caudalmiddlefrontal",
                  "left-vol-cuneus","left-vol-entorhinal","left-vol-fusiform","left-vol-inferiorparietal",
                  "left-vol-inferiortemporal","left-vol-isthmuscingulate","left-vol-lateraloccipital",
                  "left-vol-lateralorbitofrontal","left-vol-lingual","left-vol-medialorbitofrontal",
                  "left-vol-middletemporal","left-vol-parahippocampal","left-vol-paracentral",
                  "left-vol-parsopercularis","left-vol-parsorbitalis","left-vol-parstriangularis",
                  "left-vol-pericalcarine","left-vol-postcentral","left-vol-posteriorcingulate",
                  "left-vol-precentral","left-vol-precuneus","left-vol-rostralanteriorcingulate",
                  "left-vol-rostralmiddlefrontal","left-vol-superiorfrontal","left-vol-superiorparietal",
                  "left-vol-superiortemporal","left-vol-supramarginal","left-vol-frontalpole",
                  "left-vol-temporalpole","left-vol-transversetemporal","left-vol-insula","right-vol-bankssts",
                  "right-vol-caudalanteriorcingulate","right-vol-caudalmiddlefrontal","right-vol-cuneus",
                  "right-vol-entorhinal","right-vol-fusiform","right-vol-inferiorparietal",
                  "right-vol-inferiortemporal","right-vol-isthmuscingulate","right-vol-lateraloccipital",
                  "right-vol-lateralorbitofrontal","right-vol-lingual","right-vol-medialorbitofrontal",
                  "right-vol-middletemporal","right-vol-parahippocampal","right-vol-paracentral",
                  "right-vol-parsopercularis","right-vol-parsorbitalis","right-vol-parstriangularis",
                  "right-vol-pericalcarine","right-vol-postcentral","right-vol-posteriorcingulate",
                  "right-vol-precentral","right-vol-precuneus","right-vol-rostralanteriorcingulate",
                  "right-vol-rostralmiddlefrontal","right-vol-superiorfrontal","right-vol-superiorparietal",
                  "right-vol-superiortemporal","right-vol-supramarginal","right-vol-frontalpole",
                  "right-vol-temporalpole","right-vol-transversetemporal","right-vol-insula",
                  "right-svol-Thalamus","right-svol-Caudate","right-svol-Putamen","right-svol-Pallidum",
                  "right-svol-Hippocampus","right-svol-Amygdala","right-svol-Accumbens","left-svol-Thalamus",
                  "left-svol-Caudate","left-svol-Putamen","left-svol-Pallidum","left-svol-Hippocampus",
                  "left-svol-Amygdala","left-svol-Accumbens"]

In [ ]:
x = torch.rand(5, 3)
print(type(x))
x = torch.tensor([5.5, 3])
print(type(x))

In [ ]:
volner_tensor = torch.tensor(volner_vector)
volner_tensor2 = torch.from_numpy(volner_vector)